In [1]:
import pandas as pd

In [2]:
#time is in the form XXH:XXm:XXs, so we need to convert it to seconds
def convert_time(time):
    hours = int(time[0:2])
    mins = int(time[4:6])
    secs = int(time[8:10])
    return hours*3600 + mins*60 + secs

In [6]:
phases = pd.read_excel('phases.xlsx', engine='openpyxl')
print(phases.shape)

(767, 13)


In [118]:
#sort vid_id in reverse alphabetical order, in order to include yale videos in training set
phases = phases.sort_values(by=['vid_id'], ascending=False)
phases.head()

,vid_id,phase,time_start,time_end,labeler,labeler_2,notes
665,yale_9,hiatal_dissec,00H:18m:28s,00H:21m:20s,sp,NaN,NaN
671,yale_9,hiatal_dissec,00H:41m:33s,00H:43m:43s,sp,NaN,NaN
659,yale_9,exposure,00H:00m:00s,00H:02m:06s,sp,NaN,NaN
660,yale_9,sac_reduction,00H:02m:07s,00H:06m:26s,sp,NaN,NaN
661,yale_9,hiatal_dissec,00H:06m:27s,00H:07m:56s,sp,NaN,NaN


In [119]:
phases.head()

,vid_id,phase,time_start,time_end,labeler,labeler_2,notes
665,yale_9,hiatal_dissec,00H:18m:28s,00H:21m:20s,sp,NaN,NaN
671,yale_9,hiatal_dissec,00H:41m:33s,00H:43m:43s,sp,NaN,NaN
659,yale_9,exposure,00H:00m:00s,00H:02m:06s,sp,NaN,NaN
660,yale_9,sac_reduction,00H:02m:07s,00H:06m:26s,sp,NaN,NaN
661,yale_9,hiatal_dissec,00H:06m:27s,00H:07m:56s,sp,NaN,NaN


In [120]:
#for vid_4, keep only labeler 'kc'
print(phases.shape)
non_kc_vid_4 = phases[phases['vid_id'] == 'vid_4']
non_kc_vid_4 = non_kc_vid_4[non_kc_vid_4['labeler'] != 'kc']
phases = phases.drop(non_kc_vid_4.index)
phases.reset_index(drop=True, inplace=True)
print(phases.shape)

(767, 7)
(707, 7)


In [121]:
#rename 'sac_reduction' phase to 'hiatal_dissec'
phases.loc[phases['phase'] == 'sac_reduction', 'phase'] = 'hiatal_dissec'

In [122]:
#get the number of unique videos
len(phases['vid_id'].unique())

37

In [123]:
phases['labeler'].value_counts()

kc     251
sp     177
ck     161
AB     117
ck       1
Name: labeler, dtype: int64

In [124]:
#strip leading and trailing whitespace from the time_start and time_end columns
phases['time_start'] = phases['time_start'].str.strip()
phases['time_end'] = phases['time_end'].str.strip()

In [126]:
#for each row in the dataframe, make sure that time_start and time_end are integers
for i in range(len(phases)):
    try:
        start1 = convert_time(phases['time_start'][i])
        end1 = convert_time(phases['time_end'][i])
    except:
        print(i)
        print(phases['time_start'][i])
        print(phases['time_end'][i])

In [127]:
phase_list = phases['phase'].unique()
phase_list

array(['hiatal_dissec', 'exposure', 'other', 'oob', 'eso_mob',
       'hiatal_repair', 'wrap', 'sac_excision', 'fundus_mob',
       'peg_placement'], dtype=object)

In [128]:
#check the total amount of time in the dataset
total_time = 0
for i in range(len(phases)):
    start1 = convert_time(phases['time_start'][i])
    end1 = convert_time(phases['time_end'][i])
    total_time += (end1 - start1)

#get the total time in hours, minutes, and seconds
hours = total_time // 3600
mins = (total_time % 3600) // 60
secs = (total_time % 3600) % 60
print(hours, 'hrs,', mins, 'mins,', secs, 'secs')

48 hrs, 12 mins, 8 secs


In [129]:
print(len(phases))

707


In [130]:
phases['path'] = np.NaN
#if vid_id contains 'vid', then path starts with 'unc/raw_ids', elif vid_id contains 'yale', then path starts with 'yale/raw_ids'
for i, row in phases.iterrows():
    if 'vid' in row['vid_id']:
        phases.loc[i, 'path'] = 'unc/raw_ids/' + row['vid_id'] + '.mp4'
    elif 'yale' in row['vid_id']:
        phases.loc[i, 'path'] = 'yale/raw_ids/' + row['vid_id'] + '.mp4'
    else:
        print('error')

In [131]:
#for each video, check if vid_{}_v2.mp4 exists, if it does, then replace the path with that
for i, row in phases.iterrows():
    if os.path.exists('unc/raw_ids/' + row['vid_id'] + '_v2.mp4'):
        phases.loc[i, 'path'] = 'unc/raw_ids/' + row['vid_id'] + '_v2.mp4'
    elif os.path.exists('yale/raw_ids/' + row['vid_id'] + '_v2.mp4'):
        phases.loc[i, 'path'] = 'yale/raw_ids/' + row['vid_id'] + '_v2.mp4'
    else:
        pass
phases.head()

,vid_id,phase,time_start,time_end,labeler,labeler_2,notes,path
0,yale_9,hiatal_dissec,00H:18m:28s,00H:21m:20s,sp,NaN,NaN,yale/raw_ids/yale_9.mp4
1,yale_9,hiatal_dissec,00H:41m:33s,00H:43m:43s,sp,NaN,NaN,yale/raw_ids/yale_9.mp4
2,yale_9,exposure,00H:00m:00s,00H:02m:06s,sp,NaN,NaN,yale/raw_ids/yale_9.mp4
3,yale_9,hiatal_dissec,00H:02m:07s,00H:06m:26s,sp,NaN,NaN,yale/raw_ids/yale_9.mp4
4,yale_9,hiatal_dissec,00H:06m:27s,00H:07m:56s,sp,NaN,NaN,yale/raw_ids/yale_9.mp4


In [132]:
#find values in 'path' that are not strings
phases[phases['path'].apply(lambda x: type(x) != str)]

,vid_id,phase,time_start,time_end,labeler,labeler_2,notes,path


In [76]:
#split the number of phases into training, validation, and test sets
train_range = int(len(phases) * 0.8)
val_range = int(len(phases) * 0.9)
test_range = len(phases)
print(train_range, val_range, test_range)

532 598 665


In [79]:
#delete any existing frames
if os.path.exists('frames'):
    shutil.rmtree('frames')

for i in range(len(phases)):
    if i < train_range:
        vid_id = phases['vid_id'][i]
        vid_fname = phases['path'][i]
        phase = phases['phase'][i]
        time_start = phases['time_start'][i]
        time_end = phases['time_end'][i]
        time_start_sec = convert_time(time_start)
        time_end_sec = convert_time(time_end)
        print(vid_fname)
        #if the phase is 'other', then skip it
        if phase == 'other':
            continue
        #if the phase is not 'oob', then add a 4 second buffer to the start and end times
        elif phase != 'oob':
            time_start_sec += 4
            time_end_sec -= 4
            for i in range(time_start_sec, time_end_sec):
                if i%10 == 0:
                    cap = cv2.VideoCapture(vid_fname)
                    cap.set(cv2.CAP_PROP_POS_MSEC, i*1000)
                    ret, frame = cap.read()
                    if ret:
                        if not os.path.exists('frames/train/{}'.format(phase)):
                            os.makedirs('frames/train/{}'.format(phase))
                        cv2.imwrite('frames/train/{}/{}_{}.jpg'.format(phase, vid_id, i), frame)
                else:
                    continue
                cap.release()
        #if the phase is 'oob', then don't add a buffer
        elif phase == 'oob':
            for i in range(time_start_sec, time_end_sec):
                if i%10 == 0:
                    cap = cv2.VideoCapture(vid_fname)
                    cap.set(cv2.CAP_PROP_POS_MSEC, i*1000)
                    ret, frame = cap.read()
                    if ret:
                        if not os.path.exists('frames/train/{}'.format(phase)):
                            os.makedirs('frames/train/{}'.format(phase))
                        cv2.imwrite('frames/train/{}/{}_{}.jpg'.format(phase, vid_id, i), frame)
                else:
                    continue
                cap.release()
    #validation set
    elif i >= train_range and i < val_range:
        vid_id = phases['vid_id'][i]
        vid_fname = phases['path'][i]
        phase = phases['phase'][i]
        time_start = phases['time_start'][i]
        time_end = phases['time_end'][i]
        time_start_sec = convert_time(time_start)
        time_end_sec = convert_time(time_end)
        print(vid_fname)
        #if the phase is 'other', then skip it
        if phase == 'other':
            continue
        #if the phase is not 'oob', then add a 4 second buffer to the start and end times
        elif phase != 'oob':
            time_start_sec += 4
            time_end_sec -= 4
            for i in range(time_start_sec, time_end_sec):
                if i%10 == 0:
                    cap = cv2.VideoCapture(vid_fname)
                    cap.set(cv2.CAP_PROP_POS_MSEC, i*1000)
                    ret, frame = cap.read()
                    if ret:
                        if not os.path.exists('frames/val/{}'.format(phase)):
                            os.makedirs('frames/val/{}'.format(phase))
                        cv2.imwrite('frames/val/{}/{}_{}.jpg'.format(phase, vid_id, i), frame)
                else:
                    continue
                cap.release()
        #if the phase is 'oob', then don't add a buffer
        elif phase == 'oob':
            for i in range(time_start_sec, time_end_sec):
                if i%10 == 0:
                    cap = cv2.VideoCapture(vid_fname)
                    cap.set(cv2.CAP_PROP_POS_MSEC, i*1000)
                    ret, frame = cap.read()
                    if ret:
                        if not os.path.exists('frames/val/{}'.format(phase)):
                            os.makedirs('frames/val/{}'.format(phase))
                        cv2.imwrite('frames/val/{}/{}_{}.jpg'.format(phase, vid_id, i), frame)
                else:
                    continue
                cap.release()
    elif i >= val_range:
        vid_id = phases['vid_id'][i]
        vid_fname = phases['path'][i]
        phase = phases['phase'][i]
        time_start = phases['time_start'][i]
        time_end = phases['time_end'][i]
        time_start_sec = convert_time(time_start)
        time_end_sec = convert_time(time_end)
        print(vid_fname)
        #if the phase is 'other', then skip it
        if phase == 'other':
            continue
        #if the phase is not 'oob', then add a 4 second buffer to the start and end times
        elif phase != 'oob':
            time_start_sec += 4
            time_end_sec -= 4
            for i in range(time_start_sec, time_end_sec):
                if i%10 == 0:
                    cap = cv2.VideoCapture(vid_fname)
                    cap.set(cv2.CAP_PROP_POS_MSEC, i*1000)
                    ret, frame = cap.read()
                    if ret:
                        if not os.path.exists('frames/test/{}'.format(phase)):
                            os.makedirs('frames/test/{}'.format(phase))
                        cv2.imwrite('frames/test/{}/{}_{}.jpg'.format(phase, vid_id, i), frame)
                else:
                    continue
                cap.release()
        #if the phase is 'oob', then don't add a buffer
        elif phase == 'oob':
            for i in range(time_start_sec, time_end_sec):
                if i%10 == 0:
                    cap = cv2.VideoCapture(vid_fname)
                    cap.set(cv2.CAP_PROP_POS_MSEC, i*1000)
                    ret, frame = cap.read()
                    if ret:
                        if not os.path.exists('frames/test/{}'.format(phase)):
                            os.makedirs('frames/test/{}'.format(phase))
                        cv2.imwrite('frames/test/{}/{}_{}.jpg'.format(phase, vid_id, i), frame)
                else:
                    continue
                cap.release()

yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_9.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_8.mp4
yale/raw_ids/yale_7.mp4
yale/raw_ids/yale_7.mp4
yale/raw_ids/yale_7.mp4
yale/raw_ids/yale_7.mp4
yale/raw_ids/yale_7.mp4
yale/raw_ids/yale_7.mp4
yale/raw_ids/yal

In [80]:
#count how many frames have been generated for each phase and train/test set
count_df = pd.DataFrame(columns=['phase', 'train_test', 'count'])
index_count = 0
for phase in os.listdir('frames/train'):
    # count_df = count_df.append({'phase': phase, 'train_test': 'train', 'count': len(os.listdir('frames/train/{}'.format(phase)))}, ignore_index=True)
    count_df = pd.concat([count_df, pd.DataFrame({'phase': phase, 'train_test': 'train', 'count': len(os.listdir('frames/train/{}'.format(phase)))}, index=[index_count])], ignore_index=False)
    index_count += 1
for phase in os.listdir('frames/test'):
    # count_df = count_df.append({'phase': phase, 'train_test': 'test', 'count': len(os.listdir('frames/test/{}'.format(phase)))}, ignore_index=True)
    count_df = pd.concat([count_df, pd.DataFrame({'phase': phase, 'train_test': 'test', 'count': len(os.listdir('frames/test/{}'.format(phase)))}, index=[index_count])], ignore_index=False)
    index_count += 1
for phase in os.listdir('frames/val'):
    count_df = pd.concat([count_df, pd.DataFrame({'phase': phase, 'train_test': 'val', 'count': len(os.listdir('frames/val/{}'.format(phase)))}, index=[index_count])], ignore_index=False)
    index_count += 1


In [81]:
count_df

,phase,train_test,count
0,hiatal_dissec,train,2013
1,oob,train,462
2,wrap,train,2860
3,eso_mob,train,2147
4,hiatal_repair,train,1562
5,fundus_mob,train,676
6,exposure,train,688
7,peg_placement,train,249
8,sac_excision,train,580
9,hiatal_dissec,test,177
